In [5]:
import requests
from time import sleep, time
from tqdm import tqdm
from bs4 import BeautifulSoup 
import pandas as pd

ROOT_URL = 'http://www.vanfun.net'

In [10]:
def get_listing_data(type_code, endpage):
    '''
    type_code 1 = house
    type_code 2 = townhouse
    type_code 3 = apartment
    '''
    all_page_response = []
    for page in tqdm(range(1,endpage)): 
        req = requests.get(ROOT_URL+'/house-{}-0-0-0-0-0-0-0-0-0-{}-0-0-0-0-0-0-0-0.aspx'.format(type_code,page))
        temp_soup = BeautifulSoup(req.text, "lxml")
        # 从第三个开始 前面两个是广告
        listing_div_list = temp_soup.find('dl',{"id":"goodsList"}).find_all("dd",title =True)[3:]
        all_page_response.append(listing_div_list)
        #sleep(1)
    return all_page_response


def get_info_per_page(listing_page):
    temp_page = pd.DataFrame()
    for house in listing_page:
        temp = pd.DataFrame({'address': house["title"],
                             'id': house.find_all('a')[0]['data-code'],
                             'house__detail_link':ROOT_URL+house.find_all('a')[0]['href'],
                             'house_config':house.find('span',{"class":"orange"}).text,
                             'list_comapny':house.find('span',{"class":"listingof"}).text,
                             'price_cad':house.find('span',{"class":"price"}).text,
                             'price_rmb':house.find('span',{"class":"price_rmb"}).text
                            }, index=[0])

        temp_page = pd.concat([temp_page, temp])
    return temp_page


def save_to_excel(type_code,endpage, reponses):

    total_listing = pd.DataFrame()
    chunk_of_df = []
    for page in tqdm(range(endpage-1)):
        chunk_of_df.append(get_info_per_page(reponses[page]))
    total_listing = pd.concat(chunk_of_df,ignore_index=True)
    total_listing.to_excel('total_vancouver_{}_data.xlsx'.format(type_code))

In [7]:
townhouse_data = get_listing_data(2,267)
save_to_excel(2,266, townhouse_data)

100%|██████████| 266/266 [00:04<00:00, 59.35it/s]


In [11]:
apartment_data = get_listing_data(3,542)
save_to_excel(3,542, apartment_data)

100%|██████████| 541/541 [00:09<00:00, 59.08it/s]
